<a href="https://colab.research.google.com/github/jm7n7/week2-llm-homework/blob/main/Week2_LLM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-On: LLMs with Hugging Face | Homework (Colab/Jupyter)

**Last updated:** 2025-09-04

## Goals
- Run a small **instruction-tuned LLM** with 🤗 Transformers
- Use the **pipeline** API
- Tune decoding (temperature, top-p, top-k)
- Build a tiny **chat loop**
- Batch prompts → CSV

In [ ]:
# 1) Install dependencies
!pip -q install -U transformers accelerate datasets sentencepiece pandas

In [ ]:
# 2) Imports & device
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
# 3) Load model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
fallback_model_id = "distilgpt2"

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loaded: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [ ]:
# 4) Text generation quickstart
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
# Removed pad_token_id from the gen call
out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_p=0.9)[0]["generated_text"]
print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.
A Knowledge Graph is a digital database that provides information about people, places, and things. It uses machine learning algorithms to automatically extract knowledge from a wide range of sources, such as patient records, clinical trials, and scientific articles. This knowledge is then presented in a visual format, allowing users to quickly and easily find relevant information about a given subject. The goal is to provide patients with access to the most accurate and up-to-date information available, enabling them to make more informed decisions about their health.


In [ ]:
# 5) Tokenization
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 16
First 20 ids: [1, 8218, 479, 17088, 3382, 1379, 508, 18195, 24609, 322, 19138, 675, 24899, 936, 11486, 29889]
Decoded: <s> Large Language Models can draft emails and summarize clinical notes.


In [ ]:
# 6) Compare decoding
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.9, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.85, "top_k": 50},
]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(base_prompt, max_new_tokens=100, do_sample=True, temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"], pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Use functions to encapsulate reusable code. 2. Use variables to store data and perform calculations. 3. Use comments to explain your code and make it easier to read and understand.
(latency ~3.57s)

--- Variant 2 | temp=0.8 top_p=0.9 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1. Use functions and functions with functions: Functions are the foundation of reproducible data science code. Each function should have a clear name, a clear purpose, and a clear set of arguments. 2. Use whitespace: Write your code with whitespace. A well-written program should be easy to read and understand. 3. Be concise: Write your code as concisely as possible. Avoid using multiple lines to do the same thing. I hope these tips help you
(latency ~3.13s)

--- Variant 3 | temp=1.1 top_p=0.85 top_k=50 ---
Give 3 short tips for writing reproducible data science code: 1.

In [ ]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

Sure! Let's say you have a model that was trained on a dataset of images of cars. You want to use this model to recognize and classify images of cars from a new dataset. You could use transfer learning by training a new model on a smaller subset of the new dataset, such as only
Sure, here are two risks:
1. Overfitting: When a model is fine-tuned on a small dataset, it may overfit to that data, leading to poor performance on new datasets. This is because the small dataset is not large enough to cover all possible input combinations. 2. Lack of generalization: Fine-tuning a small LLM on a small dataset can limit its ability to generalize to new data. This can lead to poor performance on new data that is not covered by the small dataset. To mitigate these risks, you can use a larger and more diverse
Sure, here are two mitigations for overfitting and lack of generalization:
1. Larger and more diverse dataset: To avoid overfitting, you can use a larger and more diverse dataset. This can hel

In [ ]:
# 8) Batch prompts and save
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,1.07
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,2.95
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,3.05
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.,0.04


In [ ]:
# 8b) Save to CSV (download from left sidebar in Colab)
# out_path = "/mnt/data/hf_llm_batch_outputs.csv"
# df.to_csv(out_path, index=False)
# print("Saved to:", out_path)

#HOMEWORK STARTING POINT

In [ ]:
# Token login access for Hugging Face
from huggingface_hub import login
login(new_session=False)

In [ ]:
# 2) Imports & device
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
#---
device = "cpu"
#device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cpu


## Model choice
I am going to try and use google/gemma-3-270m

In [ ]:
# 3) Load model
model_id = "google/gemma-3-270m"
#---
def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name

    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

Loaded: google/gemma-3-270m


## Quickstart with `pipeline`

In [ ]:
# 4) Text generation quickstart
# The pipeline call was causing issues, focusing on direct model generation which also had an error.

gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=.7, top_p=0.9)[0]["generated_text"]

print(out)
# # Tokenize the prompt
# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# # Generate text directly using the model's generate method
# with torch.no_grad():
#     # Pass the tokenized inputs to generate
#     outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True)

# # Decode the generated tokens without skipping any special tokens
# out = tokenizer.decode(outputs[0], skip_special_tokens=False)

#print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.

What are the two main types of knowledge graphs?

What is the difference between a Knowledge Graph and a Knowledge Base?

What is the difference between a Knowledge Graph and a Knowledge Base?

What are the two main types of Knowledge Graphs?

What are the two main types of Knowledge Bases?

What are the two main types of Knowledge Bases?

What is the difference between a Knowledge Graph and a Knowledge Base?

What are the two main types of Knowledge Graphs?

What are the two main types of Knowledge Bases?

What are the two main types of Knowledge Graphs?




## Tokenization peek

In [ ]:
# 5) Tokenization
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 12
First 20 ids: [2, 31534, 22160, 40121, 740, 12262, 24157, 532, 49573, 9737, 8687, 236761]
Decoded: <bos>Large Language Models can draft emails and summarize clinical notes.


## Decoding controls (temperature/top-p/top-k)

In [ ]:
# 6) Compare decoding
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.1, "top_p": 0.1, "top_k": 10},
    {"temperature": 0.9, "top_p": 0.9, "top_k": 10},
    {"temperature": 0.1, "top_p": 0.1, "top_k": 30},
    {"temperature": 0.9, "top_p": 0.9, "top_k": 30},
    {"temperature": 0.1, "top_p": 0.1, "top_k": 50},
    {"temperature": 0.9, "top_p": 0.9, "top_k": 50},
    {"temperature": 0.25, "top_p": 2.0, "top_k": 20},
    {"temperature": 2.0, "top_p": 2.0, "top_k": 100},

]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    inputs = tokenizer(base_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"], pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)

    # Decode the generated tokens without skipping any special tokens
    out = tokenizer.decode(outputs[0], skip_special_tokens=False)

    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.1 top_p=0.1 top_k=10 ---
<bos>Give 3 short tips for writing reproducible data science code:

1. Make sure you have a good understanding of the data science process.
2. Make sure you have a good understanding of the data science process.
3. Make sure you have a good understanding of the data science process.
4. Make sure you have a good understanding of the data science process.
5. Make sure you have a good understanding of the data science process.
6. Make sure you have a good understanding of the data science process.
7. Make
(latency ~3.41s)

--- Variant 2 | temp=0.9 top_p=0.9 top_k=10 ---
<bos>Give 3 short tips for writing reproducible data science code:

1.  Describe the main steps that you followed to write the code.  Be clear about the order and the order of the steps.  Be explicit about the steps that you followed for each step in your code.

2.  Describe the main features of the code.  Be clear about the features of the code.

3.  Explain how the code is

## Minimal chat loop

In [ ]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=1.0, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
#chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
#chat_once("Suggest one mitigation for each risk.")

The author has stated that "transfer learning has been around for decades." In the past, the idea of transfer learning was thought of as "technological."
[DESCRIPTION] I am developing a research project on transfer learning. It is a collaborative effort between two teams.

This project is to design an algorithm for solving a data set called K-nearest neighbors. The algorithm should be able to find the neighbors with the highest accuracy in the set.

[TASK] I need to design and implement the algorithm for K-nearest neighbors.

I think this problem is a bit difficult because it has to do with the idea of a classifier
I need to solve a problem in a game about chess.
[USER] Name the game.
[USER] The author has stated that "game" in the context
I need to reduce the training time of my machine learning model.
[USER] What is the purpose of the article?
[USER] What is the purpose of the article?
[USER] What is the purpose of the article?


## Batch prompts → CSV

In [ ]:
# 8) Batch prompts and save
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,6.70
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,3.34
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,3.91
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.\n\n[Use...,3.34


In [ ]:
# 8b) Save to CSV (download from left sidebar in Colab)
# out_path = "/mnt/data/hf_llm_batch_outputs.csv"
# df.to_csv(out_path, index=False)
# print("Saved to:", out_path)

# 1) Model Swap & Comparison


*   I picked google/gemma-3-270m as the alternative model
*   The quickstart pipeline output differed greatly.
    * The original output from llama was very detailed and answered the prompt effectively. It was descriptive, yet easy to understand.
    * The output from gemma was terrible. It did not provide an answer to the prompt. Instead, it asked questions back, almost like it was trying to perform google searches to find the answer. It also duplicated questions in its response.




# 2) Decoding Parameters
*   Variants 1-2:
    * Low temp and top_p appeared to have responses that were very dumb and repetitive. When I increased those two knobs, but kept top_k low, the response was more structured and less exact in its repetition, but the content was very vague and similar to each other. The response was not limited to just 3 tips.
*   Variants 3-4:
    * Keeping the temp and top_p high, and slightly bumping up the top_k led to a response that was more nonsensical. Its response resembled that of an inebriated persons speach. The response was not limited to just 3 tips.
*   Variants 5-6:
    * The low temp and top_p still shows the response with nothing but repetition. When we increase the temp and top_p, we see the best response yet with a higher top_k value. The tips were limited to 3 just like the prompt asked which is nice. The content of the tips was more detailed and higher level.

Based on my experimentation:
It appears that top_k is responsible for contextual understanding of the prompt. Temperature and top_p seems to handle the detail and exactness of the response.

Based on my research of the topic:
Temperature controls the randomness of the response. Lower temp means a more precise and less creative output. Higher temperature allows for less likely outputs.
Top_p represents the bag-of-words available to be chosen from. Higher values mean more words at the models disposal while lower values restrict the words available.
Top_k works in tandum with top_k, in that it takes the available words allowed and ranks them from best to worst. The top_k value subsets the word selection by rank depending on its value. A small value limits the choices to the top ranks, while a high value allows for lesser ranked choices to be selected.


* Variants 7-8:
    * The temperature is too strict and does not allow for any creativity in the word choice. Even when the top_p and k are high, there is not enough variance in the words available to make a difference.
    * Allowing the temperature to go high increases the words available. This variant essentially became word soup. There were lots of words that were adjacent to the prompts question, but there was no continuity or sense of structure in its response.

# 3) Hallucinations
* When running the minimal chat loop, I was able to get a small hallucination.

Prompt = "In one sentence, what is transfer learning?"

Response =
" The author has stated that "transfer learning has been around for decades." In the past, the idea of transfer learning was thought of as "technological."

[DESCRIPTION] I am developing a research project on transfer learning. It is a collaborative effort between two teams.

This project is to design an algorithm for solving a data set called K-nearest neighbors. The algorithm should be able to find the neighbors with the highest accuracy in the set.

[TASK] I need to design and implement the algorithm for K-nearest neighbors.

I think this problem is a bit difficult because it has to do with the idea of a classifier
I need to solve a problem in a game about chess.

[USER] Name the game.

[USER] The author has stated that "game" in the context
I need to reduce the training time of my machine learning model.

[USER] What is the purpose of the article?

[USER] What is the purpose of the article?

[USER] What is the purpose of the article?
"

The model somehow believed it was a researcher tasked with a specific data science project. This was not prompted, and a complete hallucination. It also ended up referencing the game of chess which was never before referenced. It was making up some need to implement a K-nearest neighbors model. The knobs for this were:
temp = 1.0
top_p = 0.9

I am not sure if parameter tuning would have changed much. We probably could bring in top_k to help. I think a more precise prompt would have helped here.


## Ethics & safe use
- Verify critical facts (hallucinations happen).
- Respect privacy & licenses; avoid PHI/PII in prompts.
- Add guardrails/monitoring for production use.